In [4]:
pip install yahoo-fin

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install azure.eventhub

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import json
from yahoo_fin import stock_info as si
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
from azure.eventhub.exceptions import EventHubError
import asyncio
from datetime import datetime

In [11]:
connection_str = 'Endpoint=sb://stockfeeddatademoxhll.servicebus.windows.net/;SharedAccessKeyName=RootUserAccessPolicy;SharedAccessKey=HqI4+Cq0l6h4RLQ/iINmgQZ6+SWK0/W/q1aRqI+tq60=;EntityPath=pythonfeeddata'
eventhub_name = 'pythonfeeddata'

In [12]:
test = si.get_quote_data('nvda')

In [13]:
test_df = pd.DataFrame([test],columns=test.keys())[['regularMarketTime','regularMarketPrice','marketCap','exchange','averageAnalystRating']]
datetime.fromtimestamp(test_df['regularMarketTime'])
test_df['averageAnalystRating'].str.split(' - ',1,expand=True)
test_df.apply(lambda row: "$" + str(round(row['marketCap']/100000000000,2)) + 'MM',axis=1)

C:\Users\FANGLI~1\AppData\Local\Temp/ipykernel_11728/1475773408.py:2: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  datetime.fromtimestamp(test_df['regularMarketTime'])


0    $3.3MM
dtype: object

C:\Users\FANGLI~1\AppData\Local\Temp/ipykernel_11728/1124443489.py:5: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  stock_dataframe['regularMarketTime'] = datetime.fromtimestamp(stock_dataframe['regularMarketTime'])
C:\Users\FANGLI~1\AppData\Local\Temp/ipykernel_11728/1124443489.py:14: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  return stock_dataframe.to_dict('record')


Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs


In [14]:
def get_data_for_stock(stock):
    stock_pull = si.get_quote_data(stock)
    stock_dataframe = pd.DataFrame([stock_pull],columns=stock_pull.keys())[['regularMarketTime','regularMarketPrice','marketCap','exchange','averageAnalystRating']]
    
    stock_dataframe['regularMarketTime'] = datetime.fromtimestamp(stock_dataframe['regularMarketTime'])
    stock_dataframe['regularMarketTime'] = stock_dataframe['regularMarketTime'].astype(str)
    
    stock_dataframe[['AnalystRating','AnalystBuySell']] = stock_dataframe['averageAnalystRating'].str.split(' - ', 1,expand=True)
    
    stock_dataframe.drop('averageAnalystRating',axis = 1, inplace=True)
    
    stock_dataframe['MarketCapInTrill$$'] = stock_dataframe.apply(lambda row: "$" + str(round(row['marketCap']/1000000000000,2)) + 'MM',axis=1)
    
    return stock_dataframe.to_dict('record')

In [15]:
datetime.now()

datetime.datetime(2022, 9, 22, 0, 53, 15, 544407)

In [16]:
get_data_for_stock('aapl')

C:\Users\FANGLI~1\AppData\Local\Temp/ipykernel_11728/1124443489.py:5: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  stock_dataframe['regularMarketTime'] = datetime.fromtimestamp(stock_dataframe['regularMarketTime'])
C:\Users\FANGLI~1\AppData\Local\Temp/ipykernel_11728/1124443489.py:14: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  return stock_dataframe.to_dict('record')


[{'regularMarketTime': '2022-09-21 16:00:04',
  'regularMarketPrice': 153.72,
  'marketCap': 2470403375104,
  'exchange': 'NMS',
  'AnalystRating': '1.9',
  'AnalystBuySell': 'Buy',
  'MarketCapInTrill$$': '$2.47MM'}]

In [17]:
async def run():
    # Create a producer client to send messages to the event hub.
    # Specify a connection string to your event hubs namespace and
    # the event hub name.
    while True:
        await asyncio.sleep(5)
        producer = EventHubProducerClient.from_connection_string(conn_str=connection_str, eventhub_name=eventhub_name)
        async with producer:
            # Create a batch.
            event_data_batch = await producer.create_batch()

            # Add events to the batch.
            event_data_batch.add(EventData(json.dumps(get_data_for_stock('aapl'))))

            # Send the batch of events to the event hub.
            await producer.send_batch(event_data_batch)
            print('Success sent to Azure Event Hubs')
            
loop=asyncio.get_event_loop()
try:
    asyncio.ensure_future(run())
    loop.run_forever()
except KeyboardInterrupe:
    pass
finally:
    print("ClossingLoopNow")
    loop.close()

ClossingLoopNow


RuntimeError: Cannot close a running event loop

C:\Users\FANGLI~1\AppData\Local\Temp/ipykernel_11728/1124443489.py:5: DeprecationWarning: an integer is required (got type Series).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  stock_dataframe['regularMarketTime'] = datetime.fromtimestamp(stock_dataframe['regularMarketTime'])
C:\Users\FANGLI~1\AppData\Local\Temp/ipykernel_11728/1124443489.py:14: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  return stock_dataframe.to_dict('record')


Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success se

Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success sent to Azure Event Hubs
Success se